# Import the libraries

In [1]:
#importing libraries
import open3d as o3d #pip install open3d
import os

import numpy as np #pip install numpy
import matplotlib.pyplot as plt #pip install matplotlib

from scipy.spatial import cKDTree #pip install scipy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Define the folder path where the obj files are stored, Also optionally another folder path can be defined to save the final results into it

In [2]:
folder_path = 'C:/Users/bb26k/Desktop/obj_data' #Folder path for where obj files are stored

output_save_folder = 'C:/Users/bb26k/Desktop/save_data' #folder path for saving the data
os.makedirs(output_save_folder, exist_ok=True) #if the folder is not exist for saving, this line of code creates the folder

# Read the obj files and import the vertex and triangle topologies into a list

In [3]:
#find all .obj files in the folder
obj_files = [f for f in os.listdir(folder_path) if f.endswith(".obj")]

In [4]:
def import_obj_files(obj_files): #function that import the obj files
    meshes = []
    for obj_file in obj_files:
        file_path = os.path.join(folder_path, obj_file)
        mesh = o3d.io.read_triangle_mesh(file_path) #reading as triangle mesh since it is indicated that triangle meshes used in orginal data
        meshes.append(mesh) #store all meshes in a single list
    return meshes
        
meshes = import_obj_files(obj_files) #output of the function is a list that contains obj file data
print('all mesh data for each object:', meshes)

all mesh data for each object: [TriangleMesh with 267 points and 526 triangles., TriangleMesh with 2168 points and 4326 triangles., TriangleMesh with 210 points and 416 triangles., TriangleMesh with 22 points and 40 triangles.]


# Identify the vertices and triangle topologies of footprint regions

In [5]:
def footprint_detection(meshes):
    filtered_meshes = []
    filtered_triangles = []

    for mesh in meshes:
        # Hold vertices into numpy array
        #such as [84987.497 446709.064 -0.075], ...
        vertices = np.asarray(mesh.vertices)
        # Hold triangle topology information in numpy array
        # such as [1 7 8],...
        triangles = np.asarray(mesh.triangles)

        selected_triangles = []
        selected_z_values = []

        for tri in triangles:
            #to find all the Z values of 3 vertices for a face topology such as [-0.73699999 -0.73699999 -0.73699999], [6.40100002 6.88999987 9.21000004], ...
            tri_z_values = vertices[tri, 2] #index0: X, index1: Y, index2: Z
            if np.all(np.isclose(tri_z_values, tri_z_values[0])):  # Check if rest of the Z values in a face list is same with the first one
                selected_triangles.append(tri) #If condition is satisfied add their vertice topology indexes into a list 
                selected_z_values.append(tri_z_values[0])  # Add z values of a triangles to a list. However, only taking one Z value for a triangle is sufficient since all the Z values of a planar triangle is same

        #convert to numpy format for some operations
        selected_triangles = np.array(selected_triangles)
        selected_z_values = np.array(selected_z_values) 

        #select all the planar triangles that has minimum Z values (each object will has an unique footprint Z value but different objects might have different Z values) 
        #IT ASSUMES ALL THE FOOTPRINT TRIANGLES ARE PLANAR TO EACH OTHER
        min_z = np.min(selected_z_values) 
        #Filter the triangles that has lowest Z values 
        #So the final_triangles array is actually the footprint triangles of each objects
        final_triangles = []
        for tri, z in zip(selected_triangles, selected_z_values):
            if np.isclose(z, min_z):
                final_triangles.append(tri)

        #covert the footprint triangle topological information to numpy format
        final_triangles = np.array(final_triangles) 

        # Create new 3D meshes for footprint region
        mesh_filtered = o3d.geometry.TriangleMesh() #create an empty triangle mesh
        mesh_filtered.vertices = o3d.utility.Vector3dVector(vertices) #fills it with all vertices in the data
        mesh_filtered.triangles = o3d.utility.Vector3iVector(final_triangles) #fills it with only our created final_triangle topology
        mesh_filtered.remove_unreferenced_vertices() # remove unreferenced (unused vertices in a triangle) vertices since all vertices are included in previous line

        filtered_meshes.append(mesh_filtered)

    return filtered_meshes

footprint_meshes = footprint_detection(meshes) #call the function, output is mesh for footprints
print('Meshes for only the footprints of each object:', footprint_meshes)

Meshes for only the footprints of each object: [TriangleMesh with 17 points and 15 triangles., TriangleMesh with 737 points and 735 triangles., TriangleMesh with 52 points and 50 triangles., TriangleMesh with 10 points and 8 triangles.]


# Identify the height values of each object

In [6]:
def height_detection(meshes):
    max_z_values_per_building = []

    for mesh in meshes:
        vertices = np.asarray(mesh.vertices)

        xy_points = vertices[:, :2]  #takes X and Y values of each points (because the condition only checked in horizontal plane)
        tree = cKDTree(xy_points) #creates tree structure for fast query

        # Find the nearest neighbors for each points
        distances, _ = tree.query(xy_points, k=2) #k=1 means only the point itself will be considered so no distance calculations will be if k selected as 1
        #find the nearest distance values to next point for each point
        nearest_distances = distances[:, 1]  #if k=2, then index0 is point itself, index1 is the nearest point

        # create the mask: if a points nearest distance is smaller than 4 meter then those points will be excluded
        mask_far_enough = nearest_distances >= 4
        # filter the vertices according to that mask
        far_enough_vertices = vertices[mask_far_enough]

        # If none of the points for an object has a point with nearest neighbor that is at least 4 meters away in the XY plane, the maximum Z value among all points will be used as the building height
        if far_enough_vertices.shape[0] == 0:
            max_z = np.max(vertices[:, 2])
        else:
            #If the object have points with a nearest neigbor that is at least 4 meter away, then use the height value as the point with highest Z value among these points
            max_z = np.max(far_enough_vertices[:, 2]) 

        max_z_values_per_building.append(max_z)

    return max_z_values_per_building

building_heights = height_detection(meshes) #call the function, output is height values per building

#print the detected height values
for value in building_heights:
        print('Building height value will be:', value)


Building height value will be: 12.00100040435791
Building height value will be: 24.031999588012695
Building height value will be: 19.937000274658203
Building height value will be: 9.79699993133545


# CONSTRUCT THE ROOFTOP VERTICES, ROOFTOP TRIANGLE TOPOLOGY AND SIDEWALL TOPOLOGY

In [12]:
def construct_rooftop_and_wall_meshes(filtered_meshes, max_z_values_per_building):
    
    complete_buildings = []

    for mesh, height in zip(filtered_meshes, max_z_values_per_building):
        base_vertices = np.asarray(mesh.vertices)
        base_triangles = np.asarray(mesh.triangles)

        top_vertices = base_vertices.copy() #copy the footprint vertices
        top_vertices[:, 2] += height #offset the copies vertices according to relavant building heights

        # STACKING THE FOOTPRINT AND ROOFTOP VERTICES
        combined_vertices = np.vstack([base_vertices, top_vertices])
        n_base = len(base_vertices) #number of footprint vertices
        #for example lets assume number of footprint vertices will be 10. Then the vertice Index of the rooftop vertices will start from 11
        
        #CONSTRUCT THE ROOFTOP TRIANGLE TOPOLOGY
        #Inverts the orientation of footprint triangles for rooftop triangles. If not so, the rooftops wont be properly visualized
        #for example instead of using triangle topology like [17,18,19], it uses [19,18,17]
        # with the "+n_base" instead of using footprint indexes like [2,1,0], it uses rooftop indexes [19,18,17]
        top_triangles = base_triangles[:, [2, 1, 0]] + n_base

        # CONSTRUCT THE SIDEWALLS TRIANGLE TOPOLOGY
        side_triangles = []
        for tri in base_triangles:
            for i in range(3):
                #Create rectangles based on footprint and rooftop vertices
                #3 different rectangle per footprint-rooftop triangle
                #For example if triangle vertice indices are [0,1,2] and rooftop indices starts from [17,18,19]
                #Then created rectangles for this triangle will be [0,1,17,18], [1,2,18,19], [2, 0, 19, 17]
                i1 = tri[i]
                i2 = tri[(i + 1) % 3]
                j1 = i1 + n_base
                j2 = i2 + n_base
                # Creates two triangle per rectangle
                side_triangles.append([i1, j1, i2])
                side_triangles.append([i2, j1, j2])

        # Stack all triangles
        all_triangles = np.vstack([base_triangles, top_triangles, side_triangles])

        # Build the triangle mesh model
        building_mesh = o3d.geometry.TriangleMesh()
        building_mesh.vertices = o3d.utility.Vector3dVector(combined_vertices)
        building_mesh.triangles = o3d.utility.Vector3iVector(all_triangles)
        
        #for colorful visualization
        building_mesh.paint_uniform_color([0.8, 0.3, 0.3])  

        complete_buildings.append(building_mesh)
    
    return complete_buildings


complete_buildings = construct_rooftop_and_wall_meshes(footprint_meshes, building_heights) #call the function, output is completed final result
print(complete_buildings)

[TriangleMesh with 34 points and 120 triangles., TriangleMesh with 1474 points and 5880 triangles., TriangleMesh with 104 points and 400 triangles., TriangleMesh with 20 points and 64 triangles.]


# VISUALIZATION OF FINAL RESULT

In [13]:
def visualization(a_mesh_result):
    o3d.visualization.draw_geometries(a_mesh_result, mesh_show_wireframe=True)
    
visualization(complete_buildings) #call the function 

# SAVE DATA OF FINAL RESULTS

In [14]:
def save_as_separate_files_per_object(complete_buildings):
    for i, mesh in enumerate(complete_buildings):
        filename = f"building_{i}.obj"
        file_path = os.path.join(output_save_folder, filename)
        o3d.io.write_triangle_mesh(file_path, mesh)
    
save_as_separate_files_per_object(complete_buildings)